In [3]:
%load_ext nb_black
%load_ext lab_black

<IPython.core.display.Javascript object>

In [1]:
# import libraries
import tensorflow as tf
import pandas as pd
from tensorflow import keras

import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

In [12]:
train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

<IPython.core.display.Javascript object>

In [19]:
train = pd.read_csv(train_file_path, sep="\t")
test = pd.read_csv(test_file_path, sep="\t")

train_data, train_labels = train.iloc[:, 1], train.iloc[:, 0]
test_data, test_labels = test.iloc[:, 1], test.iloc[:, 0]

<IPython.core.display.Javascript object>

In [39]:
key = {0: "ham", 1: "spam"}

<IPython.core.display.Javascript object>

In [37]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train_labels = le.fit_transform(train_labels)

test_labels = le.fit_transform(test_labels)

<IPython.core.display.Javascript object>

In [49]:
from keras.preprocessing import sequence
from keras.datasets.imdb import get_word_index

word_index = get_word_index()
MAXLEN = 250


def encode_text(text):
    tokens = tf.keras.preprocessing.text.text_to_word_sequence(text)
    tokens = [word_index[word] if word in word_index else 0 for word in tokens]
    return sequence.pad_sequences([tokens], MAXLEN)[0]

<IPython.core.display.Javascript object>

In [27]:
for x in range(len(train_data)):
    train_data.iloc[x] = encode_text(train_data.iloc[x])
for x in range(len(test_data)):
    test_data.iloc[x] = encode_text(test_data.iloc[x])

train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

<IPython.core.display.Javascript object>

In [30]:
reverse_word_index = {value: key for (key, value) in word_index.items()}


def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
        if num != 0:
            text += reverse_word_index[num] + " "

    return text[:-1]


print(decode_integers(train_data[0]))

ahhhh just woken up had a bad dream about u tho so i dont like u right now i didnt know anything about comedy night but i guess im up for it


<IPython.core.display.Javascript object>

In [33]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Embedding(88584, 32),
        tf.keras.layers.LSTM(32),
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)

<IPython.core.display.Javascript object>

In [34]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          2834688   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


<IPython.core.display.Javascript object>

In [35]:
model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=["acc"])

<IPython.core.display.Javascript object>

In [40]:
history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
105/105 [==============================] - 13s 102ms/step - loss: 0.2848 - acc: 0.8929 - val_loss: 0.1165 - val_acc: 0.9737
Epoch 2/10
105/105 [==============================] - 12s 113ms/step - loss: 0.0959 - acc: 0.9785 - val_loss: 0.0796 - val_acc: 0.9856
Epoch 3/10
105/105 [==============================] - 11s 107ms/step - loss: 0.0491 - acc: 0.9889 - val_loss: 0.0841 - val_acc: 0.9773
Epoch 4/10
105/105 [==============================] - 14s 129ms/step - loss: 0.0389 - acc: 0.9889 - val_loss: 0.0515 - val_acc: 0.9880
Epoch 5/10
105/105 [==============================] - 13s 120ms/step - loss: 0.0246 - acc: 0.9940 - val_loss: 0.0489 - val_acc: 0.9892
Epoch 6/10
105/105 [==============================] - 14s 132ms/step - loss: 0.0178 - acc: 0.9949 - val_loss: 0.0482 - val_acc: 0.9880
Epoch 7/10
105/105 [==============================] - 11s 100ms/step - loss: 0.0225 - acc: 0.9928 - val_loss: 0.0521 - val_acc: 0.9892
Epoch 8/10
105/105 [==============================] - 1

<IPython.core.display.Javascript object>

In [41]:
results = model.evaluate(test_data, test_labels)
print(results)


44/44 [==============================] - 1s 20ms/step - loss: 0.0456 - acc: 0.9878
[0.045554954558610916, 0.9877873659133911]


<IPython.core.display.Javascript object>

In [47]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    encoded_text = encode_text(pred_text)
    pred = np.zeros((1, 250))
    pred[0] = encoded_text
    result = model.predict(pred)
    return [result[0][0], key[round(model.predict(pred)[0][0])]]


pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

[2.8330734e-05, 'ham']


<IPython.core.display.Javascript object>

In [48]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
    test_messages = [
        "how are you doing today",
        "sale today! to stop texts call 98912460324",
        "i dont want to go. can we try it a different day? available sat",
        "our new mobile video service is live. just install on your phone to start watching.",
        "you have won £1000 cash! call to claim your prize.",
        "i'll bring it tomorrow. don't forget the milk.",
        "wow, is your arm alright. that happened to me one time too",
    ]

    test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
    passed = True

    for msg, ans in zip(test_messages, test_answers):
        prediction = predict_message(msg)
        if prediction[1] != ans:
            passed = False

    if passed:
        print("You passed the challenge. Great job!")
    else:
        print("You haven't passed yet. Keep trying.")


test_predictions()

You passed the challenge. Great job!


<IPython.core.display.Javascript object>